The idea for this excercise is to be able to load an external file, in this case the format was jsonlines, load it with python, make transformations, and have a file ready to be used in a machine learning model

import the libraries used, in this case pandas, is basically the main one, os is used to be bale to see in which file we are at so we can load the dataset

In [3]:
# machine learning model that can predict if a model is new or old based on the features 
import pandas as pd
import os

In [4]:
print(os.getcwd())

/Users/santiagoaristizabal/Learning/python/etl_course/Week_5


Loading the dataset from a json file, in this case we have to put the lines True to be able to read it

In [22]:
dataset = pd.read_json("dataset/MLA_100k.jsonlines",lines=True)

Example of the dataset

In [23]:
dataset.head()

,seller_address,warranty,sub_status,condition,seller_contact,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,...,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,geolocation,sold_quantity,available_quantity
0,"{'comment': '', 'longitude': -58.3986709, 'id'...",None,[],new,None,[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",74952096,...,active,None,NaN,NaN,1,2015-09-05 20:42:53+00:00,http://articulo.mercadolibre.com.ar/MLA-578052...,"{'latitude': -34.6280698, 'longitude': -58.398...",0,1
1,"{'comment': '', 'longitude': -58.5059173, 'id'...",NUESTRA REPUTACION,[],used,None,[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",42093335,...,active,None,NaN,NaN,1,2015-09-26 18:08:30+00:00,http://articulo.mercadolibre.com.ar/MLA-581565...,"{'latitude': -34.5935524, 'longitude': -58.505...",0,1
2,"{'comment': '', 'longitude': -58.4143948, 'id'...",None,[],used,None,[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",133384258,...,active,None,NaN,NaN,1,2015-09-09 23:57:07+00:00,http://articulo.mercadolibre.com.ar/MLA-578780...,"{'latitude': -34.6233907, 'longitude': -58.414...",0,1
3,"{'comment': '', 'longitude': -58.4929208, 'id'...",None,[],new,None,[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",143001605,...,active,None,NaN,NaN,1,2015-09-28 18:47:56+00:00,http://articulo.mercadolibre.com.ar/MLA-581877...,"{'latitude': -34.6281894, 'longitude': -58.492...",0,1
4,"{'comment': '', 'longitude': -58.5495042, 'id'...",MI REPUTACION.,[],used,None,[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",96873449,...,active,None,NaN,NaN,1,2015-08-24 22:07:20+00:00,http://articulo.mercadolibre.com.ar/MLA-576112...,"{'latitude': -34.6346547, 'longitude': -58.549...",0,1


In [24]:
print(f"Null values: \n{dataset.isna().sum()}")

Null values: 
seller_address                           0
warranty                             60896
sub_status                               0
condition                                0
seller_contact                       97781
deal_ids                                 0
base_price                               0
shipping                                 0
non_mercado_pago_payment_methods         0
seller_id                                0
variations                               0
location                                 0
site_id                                  0
listing_type_id                          0
price                                    0
attributes                               0
buying_mode                              0
tags                                     0
listing_source                           0
parent_item_id                       23011
coverage_areas                           0
category_id                              0
descriptions                            

The columns with most null values are the following in order:
-   subtitle: 100000
-   differential_pricing: 100000
-   catalog_product_id: 99989
-   original_price: 99857
-   official_store_id: 99182
-   seller_contact: 97781
-   video_id:97015
-   warranty: 60896
-   parent_item_id: 23011


We make a copy of the model, so we dont change it, in case we want to do a comparision
- For this case we are going to be saving the following columns:
    - condition -> this is the target, so we can know if its new or used
    - base_price -> the base price of the item
    - shipping -> this is a list with a dictionary, that we are going to safe exactly local_pick_up, free_shipping
    - listing_type_id -> this is a type of the seller or the item
    - price -> price of the object
    - buying_mode -> the type of the buying
    - accepts_mercadopago -> if the sellelr accepts mercadopago
    - currency_id -> this is the currency of the item, this can be from another country or currency
    - automatic_relist -> this can tell us if the item have more items in stock
    - date_created -> We can destructured the file for year, month, and day
    - status -> if the item is active, paused, or if the seller already sold it
    - initial_quantity -> initial quantity of the item
    - sold_quantity -> the items sold till the point it took the data
    - available_quantity -> available items to sell 


In [93]:
df = dataset.copy()
df = df[["condition","base_price","shipping","site_id","listing_type_id","price","buying_mode","accepts_mercadopago","currency_id","automatic_relist","date_created","status","initial_quantity","sold_quantity","available_quantity"]]

In [94]:
df.head()


,condition,base_price,shipping,site_id,listing_type_id,price,buying_mode,accepts_mercadopago,currency_id,automatic_relist,date_created,status,initial_quantity,sold_quantity,available_quantity
0,new,80.0,"{'local_pick_up': True, 'methods': [], 'tags':...",MLA,bronze,80.0,buy_it_now,True,ARS,False,2015-09-05T20:42:53.000Z,active,1,0,1
1,used,2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...",MLA,silver,2650.0,buy_it_now,True,ARS,False,2015-09-26T18:08:30.000Z,active,1,0,1
2,used,60.0,"{'local_pick_up': True, 'methods': [], 'tags':...",MLA,bronze,60.0,buy_it_now,True,ARS,False,2015-09-09T23:57:07.000Z,active,1,0,1
3,new,580.0,"{'local_pick_up': True, 'methods': [], 'tags':...",MLA,silver,580.0,buy_it_now,True,ARS,False,2015-09-28T18:47:56.000Z,active,1,0,1
4,used,30.0,"{'local_pick_up': True, 'methods': [], 'tags':...",MLA,bronze,30.0,buy_it_now,True,ARS,False,2015-08-24T22:07:20.000Z,active,1,0,1


We extract from the dictionary, the information that we want to safe in this case we only want if the item is a local_pickup, or if its free_shipping

In [95]:
def extract_shipping(dictionary):
    local_pickup = dictionary['local_pick_up']
    free_shipping = dictionary['free_shipping']

    return local_pickup, free_shipping

df[["local_pickup","free_shipping"]] = df["shipping"].apply(lambda x: pd.Series(extract_shipping(x)))




After the conversion we drop the column so we have a cleaner dataframe

In [96]:
df.drop(['shipping'],axis=1,inplace=True)

We have a column with a date, in this case we need to convert it to a datetime to be able to extract the year, month, day

In [97]:
df['date_created'] = pd.to_datetime(df['date_created'])

In [98]:
 df["year"] = df['date_created'].dt.strftime('%Y')
 df["month"] = df['date_created'].dt.strftime('%m')
 df["day"] = df['date_created'].dt.strftime('%d')

In [99]:
df.drop(['date_created'],axis=1,inplace=True)

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   condition            100000 non-null  object 
 1   base_price           100000 non-null  float64
 2   site_id              100000 non-null  object 
 3   listing_type_id      100000 non-null  object 
 4   price                100000 non-null  float64
 5   buying_mode          100000 non-null  object 
 6   accepts_mercadopago  100000 non-null  bool   
 7   currency_id          100000 non-null  object 
 8   automatic_relist     100000 non-null  bool   
 9   status               100000 non-null  object 
 10  initial_quantity     100000 non-null  int64  
 11  sold_quantity        100000 non-null  int64  
 12  available_quantity   100000 non-null  int64  
 13  local_pickup         100000 non-null  bool   
 14  free_shipping        100000 non-null  bool   
 15  year              

In [101]:
print(df.duplicated().sum())

28636


There is a total of 28636 total duplicates in the dataset

In [102]:
df = df.drop_duplicates()

before droping the duplicates we had 100000 rows after we got 71364 droping 28636 rows that were duplicates

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71364 entries, 0 to 99996
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   condition            71364 non-null  object 
 1   base_price           71364 non-null  float64
 2   site_id              71364 non-null  object 
 3   listing_type_id      71364 non-null  object 
 4   price                71364 non-null  float64
 5   buying_mode          71364 non-null  object 
 6   accepts_mercadopago  71364 non-null  bool   
 7   currency_id          71364 non-null  object 
 8   automatic_relist     71364 non-null  bool   
 9   status               71364 non-null  object 
 10  initial_quantity     71364 non-null  int64  
 11  sold_quantity        71364 non-null  int64  
 12  available_quantity   71364 non-null  int64  
 13  local_pickup         71364 non-null  bool   
 14  free_shipping        71364 non-null  bool   
 15  year                 71364 non-null  obje

In [105]:
print(f"Null values: \n{df.isna().sum()}")

Null values: 
condition              0
base_price             0
site_id                0
listing_type_id        0
price                  0
buying_mode            0
accepts_mercadopago    0
currency_id            0
automatic_relist       0
status                 0
initial_quantity       0
sold_quantity          0
available_quantity     0
local_pickup           0
free_shipping          0
year                   0
month                  0
day                    0
dtype: int64


In [107]:
df.head()

,condition,base_price,site_id,listing_type_id,price,buying_mode,accepts_mercadopago,currency_id,automatic_relist,status,initial_quantity,sold_quantity,available_quantity,local_pickup,free_shipping,year,month,day
0,new,80.0,MLA,bronze,80.0,buy_it_now,True,ARS,False,active,1,0,1,True,False,2015,09,05
1,used,2650.0,MLA,silver,2650.0,buy_it_now,True,ARS,False,active,1,0,1,True,False,2015,09,26
2,used,60.0,MLA,bronze,60.0,buy_it_now,True,ARS,False,active,1,0,1,True,False,2015,09,09
3,new,580.0,MLA,silver,580.0,buy_it_now,True,ARS,False,active,1,0,1,True,False,2015,09,28
4,used,30.0,MLA,bronze,30.0,buy_it_now,True,ARS,False,active,1,0,1,True,False,2015,08,24
